In [1]:
import torch 
from torch import nn
class DetectorPanel(nn.Module):
    def __init__(self, xy, z):
        super().__init__()
        self.xy = xy
        self.z = z
    def updatez(self, z):
        self.z = z

    def print(self):
        print(f"\t Panel has xy={self.xy.data}, z={self.z.data}")

class GDetectorPanel(nn.Module):
    def __init__(self, xy, z, globz):
        super().__init__()
        self.xy = xy
        self.z = z
        self.globalz = globz
    def updatez(self, z):
        self.z = z

    def print(self):
        print(f"\t Panel has xy={self.xy.data}, z={self.z.data+self.globalz.data}")
 
class Hodoscope(nn.Module):
    def __init__(self, xy, z):
        super().__init__()
        self.xy = nn.parameter.Parameter(data=xy, requires_grad=True)
        self.z = nn.parameter.Parameter(data=z, requires_grad=True)
        self.panels = []
        
    def generatePanels(self, relzs):
        self.positionList= [nn.parameter.Parameter(data=i, requires_grad=False) for i in relzs]
        for z in self.positionList:
            globalz=torch.nn.parameter.Parameter(data=self.z+z,requires_grad=True)
            self.panels.append(DetectorPanel(self.xy,globalz))
    def generateGPanels(self, relzs):
        self.positionList= [nn.parameter.Parameter(data=i, requires_grad=False) for i in relzs]
        for z in self.positionList:
            globalz=torch.nn.parameter.Parameter(data=self.z+z,requires_grad=True)
            self.panels.append(GDetectorPanel(self.xy,z, self.z))

    def print(self):
        print(f"Hodoscope has xy={self.xy.data}, z={self.z.data}")
            
    def printPanels(self):
        for panel in self.panels:
            panel.print()
            
            
    def modifyPosition(self, xy, z):
        #self.xy = nn.parameter.Parameter(xy)
        #self.z = nn.parameter.Parameter(z)
        self.xy.data = xy
        self.z.data = z
        #for panel, relz in zip(self.panels,self.positionList):
        #    panel.updatez(self.z+relz)
        
hod = Hodoscope(torch.Tensor([2,3]), torch.Tensor([5]))
positionList=torch.Tensor([0.,0.3,0.9])

hod.generateGPanels(positionList)
hod.print()

hod.printPanels()

hod.modifyPosition(torch.Tensor([1,2]), torch.Tensor([8]))

hod.print()
hod.printPanels()

Hodoscope has xy=tensor([2., 3.]), z=tensor([5.])
	 Panel has xy=tensor([2., 3.]), z=tensor([5.])
	 Panel has xy=tensor([2., 3.]), z=tensor([5.3000])
	 Panel has xy=tensor([2., 3.]), z=tensor([5.9000])
Hodoscope has xy=tensor([1., 2.]), z=tensor([8.])
	 Panel has xy=tensor([1., 2.]), z=tensor([8.])
	 Panel has xy=tensor([1., 2.]), z=tensor([8.3000])
	 Panel has xy=tensor([1., 2.]), z=tensor([8.9000])
